<a href="https://colab.research.google.com/github/vs412/KMeans-clustering/blob/master/Automated_sales_report_order_wise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
# !pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

start_date = '2020-01-01'
end_date = '2020-01-04'

receive_date = '2020-01-05'

sales_data = '1GlNUUsSkurloPMXlZAXY_PjE5TkDjFoH' 
pro_export_vendor = '13Q9vegTNrEFbpHCqu86ohcTGPzqpunR8' ## product export
import_ven = '1-gJKgNfOoaAYjmRJeHQw9TYrF6NsOYRz' ## import file
ware_house_ven = '1NZomAQWTB2eq3RIjITuRSWFTpxNun6SR'
drop_ship_ven = '1jbTaxZ5u4MgC4c33CSdXDhq97fcVW5pr'
closeout_sku = '1fNRz-1ND38cdsIqZljdxa6rnCvaHBeuR'
#oct sale = 1KeTHqMyvu_jLvn6Fi7Bb8uKKIugVVDwe
#nov sale = 1PZWi-RUXJFnXM_lCH1mo6iPcwJ_W22Pb
nov_sale = '1PZWi-RUXJFnXM_lCH1mo6iPcwJ_W22Pb'
last_month_sale = '1-F9cCJRXMgAZQms26r7uZosJHErMP8Ld'
monthly_sale = '1vHRDF0ciAVeNMXqPM8fbPSWhaTnPJYZX' 

sales_raw = drive.CreateFile({'id': sales_data})
pro_export_vendor = drive.CreateFile({'id': pro_export_vendor})
import_ven = drive.CreateFile({'id': import_ven})
ware_house_ven = drive.CreateFile({'id': ware_house_ven})
nov_sale = drive.CreateFile({'id': nov_sale})
last_month_sale = drive.CreateFile({'id': last_month_sale})
monthly_sale = drive.CreateFile({'id': monthly_sale})
drop_ship_ven = drive.CreateFile({'id': drop_ship_ven})
closeout_sku = drive.CreateFile({'id': closeout_sku})

# Download the file to a local disk as 'exported.xlsx'.
sales_raw.GetContentFile('Sales order.xlsx')
pro_export_vendor.GetContentFile('export.xlsx')
import_ven.GetContentFile('Import List (5).xlsx')
ware_house_ven.GetContentFile('ware_house_ven.xlsx')
nov_sale.GetContentFile('nov_sale.xlsx')
monthly_sale.GetContentFile('monthly_sale.xlsx')
last_month_sale.GetContentFile('last_month_sale.xlsx')
drop_ship_ven.GetContentFile('drop_ship_ven.xlsx')
closeout_sku.GetContentFile('closeout_sku.xlsx')

In [0]:
import pandas as pd
import numpy as np

sale_report = pd.read_excel('Sales order.xlsx',sheet_name = 0)
## reading the Casagear Data
casagear = pd.read_excel('Sales order.xlsx',sheet_name=2)
amazon = pd.read_excel('Sales order.xlsx',sheet_name=1)

In [0]:
ds_total = pd.read_excel('monthly_sale.xlsx',sheet_name=0)
ds_import = pd.read_excel('monthly_sale.xlsx',sheet_name=1)
ds_warehouse = pd.read_excel('monthly_sale.xlsx',sheet_name=2)
ds_dropship = pd.read_excel('monthly_sale.xlsx',sheet_name=3)
ds_closeout = pd.read_excel('monthly_sale.xlsx',sheet_name=4)

ds_total['Date'] = pd.to_datetime(ds_total['Date'])
ds_import['Date'] = pd.to_datetime(ds_import['Date'])
ds_warehouse['Date'] = pd.to_datetime(ds_warehouse['Date'])
ds_dropship['Date'] = pd.to_datetime(ds_dropship['Date'])
ds_closeout['Date'] = pd.to_datetime(ds_closeout['Date'])

In [23]:
ds_import['Line Total'].sum()

50073.88

In [24]:
# sale_report['Line Total'].sum()
# casagear['Total'].sum()
amazon['item-price'].sum()

5967.32

In [0]:
## Filtering the data
# sale_report['Sales Channel'].str.strip()
index = sale_report[sale_report['Sales Channel'] == 'Amazon'].index
if(len(index)!=0 ):
    sale_report.drop(index,inplace=True)
    
index = sale_report[sale_report['Sales Channel'] == 'Casagear'].index
if(len(index)!=0 ):
    sale_report.drop(index,inplace=True)
    
index = sale_report[sale_report['Sales Channel'] == 'Amazon Vendor US'].index
if(len(index)!=0 ):
    sale_report.drop(index,inplace=True)
    
index = sale_report[sale_report['Sales Channel'] == 'Amazon Vendor CA'].index
if(len(index)!=0 ):
    sale_report.drop(index,inplace=True)

index = sale_report[sale_report['Sales Channel'] == 'Amazon CA'].index
if(len(index)!=0 ):
    sale_report.drop(index,inplace=True)    
    
index = sale_report[sale_report['Sales Order Status'] == 'CANCELLED'].index

if(len(index)!=0 ):
    sale_report.drop(index,inplace=True)
    
index = sale_report[sale_report['SKU'] == 'Sample'].index
if(len(index)!=0 ):
    sale_report.drop(index,inplace=True)

index = sale_report[sale_report['SKU'] == 'sample'].index
if(len(index)!=0 ):
    sale_report.drop(index,inplace=True)
   
index = sale_report[sale_report['Receive Date'] != receive_date].index
if(len(index)!=0 ):
    sale_report.drop(index,inplace=True)

In [0]:
## reading the Amazon Data
amazon = pd.read_excel('Sales order.xlsx',sheet_name=1)
## Trimmimg the columns
amazon.columns = amazon.columns.str.strip()
## Filtering the Amazon Data
index = amazon[amazon['order-status'] == 'Cancelled'].index
if(len(index) != 0):
    amazon.drop(index,inplace=True)    
index = amazon[amazon['sales-channel'] == 'Non-Amazon'].index
if(len(index) != 0):
    amazon.drop(index,inplace=True) 
index = amazon[amazon['item-price'] == 0.0].index
if(len(index) != 0):
    amazon.drop(index,inplace=True) 
amazon['fulfillment-channel'].replace('Merchant','Amazon FBM',inplace=True)
amazon['fulfillment-channel'].replace('Amazon','Amazon FBA',inplace=True)

## Concatinating the Sales orders
meow = amazon['fulfillment-channel']
meow.reset_index(drop=True, inplace=True)
sale_report.reset_index(drop=True, inplace=True)
test = pd.concat([sale_report['Sales Channel'],meow], axis=0, ignore_index=True).iloc[0:]
new_data = pd.DataFrame(test,columns = ['Sales Channel'])

## Concatinating the Sales Channel order id
meow = amazon['amazon-order-id']
meow.reset_index(drop=True, inplace=True)
sale_report.reset_index(drop=True, inplace=True)
test = pd.concat([sale_report['Sales Channel Order Id'],meow], axis=0, ignore_index=True).iloc[0:]
new_data['Sales Channel Order Id'] = test

## Appending sku
meow = amazon['sku']
meow.reset_index(drop=True, inplace=True)
sale_report.reset_index(drop=True, inplace=True)
test = pd.concat([sale_report['SKU'],meow], axis=0, ignore_index=True).iloc[0:]
new_data['SKU'] = test

## Appending Quantity
meow = amazon['quantity']
meow.reset_index(drop=True, inplace=True)
sale_report.reset_index(drop=True, inplace=True)
test = pd.concat([sale_report['Quantity'],meow], axis=0, ignore_index=True).iloc[0:]
new_data['Quantity'] = test

## Appending Line total
meow = amazon['item-price']
meow.reset_index(drop=True, inplace=True)
sale_report.reset_index(drop=True, inplace=True)
test = pd.concat([sale_report['Line Total'],meow], axis=0, ignore_index=True).iloc[0:]
new_data['Line Total'] = test

In [0]:
## Adding the Ship to name
new_data.insert(loc=1,column = 'Ship To Name',value = sale_report['Ship To Name'])

## Inserting sales order ID
new_data.insert(loc=1,column = 'Sales Order Id',value = sale_report['Sales Order Id'])


In [0]:
## Preparing Casagear data
casagear.columns = casagear.columns.str.strip()
new_casa = pd.DataFrame(columns = new_data.columns )

new_casa.SKU = casagear['Lineitem sku']
new_casa['Sales Channel'] = 'Casagear'
new_casa['Quantity'] = casagear['Lineitem quantity']
new_casa['Sales Channel Order Id'] = casagear['Name']
new_casa['Line Total'] = casagear['Total']
new_casa['Ship To Name'] = casagear['Billing Name']


new_data = new_data.append(new_casa, ignore_index=True)

## inserting Columns Normal and Vendor
new_data.insert(loc = 3, column = 'Normal', value = '')
new_data.insert(loc = 4, column = 'Vendor', value = '')

final = new_data
#Making Columns in order
# df = new_data
# cols = list(df.columns.values)
final = final[['Sales Channel', 'Sales Order Id','Ship To Name','SKU' ,'Normal', 'Vendor','Quantity', 'Line Total','Sales Channel Order Id']]

In [0]:
final['SKU'] = final['SKU'].astype(str)

In [30]:
final.drop_duplicates()

,Sales Channel,Sales Order Id,Ship To Name,SKU,Normal,Vendor,Quantity,Line Total,Sales Channel Order Id
0,Overstock,650903.0,Todd Fortier,BM184804,,,1.0,578.82,44226159-1
1,Wayfair,650902.0,Annie Lindberg,UPT-165867,,,1.0,89.01,CS206146206
5,Hayneedle,650900.0,Michelle Wood,BM159116,,,1.0,153.60,91026038
6,Hayneedle,650899.0,Dina Verken,BM141912,,,1.0,187.15,91025375
7,Hayneedle,650898.0,Kristina Fisher,MAF-100016,,,1.0,26.49,91025378
...,...,...,...,...,...,...,...,...,...
416,Amazon FBM,NaN,NaN,GL-THROWRU,,,1.0,12.37,111-9117947-5941869
417,Amazon FBA,NaN,NaN,BM142072,,,1.0,4.90,113-3984442-4037026
418,Amazon FBM,NaN,NaN,BM01021,,,1.0,50.06,111-9698134-7278615
419,Amazon FBM,NaN,NaN,GL-1806BMSQ,,,1.0,20.43,113-4056048-7978648


In [31]:
# final = final.reset_index()
pro_exp = pd.read_excel('export.xlsx')

pro_exp['Masked SKU'] = pro_exp['Masked SKU'].astype(str)
pro_exp['SKU'] = pro_exp['SKU'].astype(str)

pro_exp.rename(columns={'Masked SKU':'SKU', 'SKU':'Normal_product'}, inplace=True)
# pro_exp.head()
final['SKU'] = final['SKU'].astype(str)
pro_exp['SKU'] = pro_exp['SKU'].astype(str)
pro_exp['Normal_product'] = pro_exp['Normal_product'].astype(str)

store = final.merge(pro_exp, on='SKU', how='left')

store = store[(store['Line Total'].notnull()) & (store['SKU']!='nan')]

store['Normal'] = store['Normal_product']
store['Vendor'] = np.nan
final = store[['Sales Channel', 'Sales Order Id','Ship To Name','SKU' ,'Normal', 'Vendor','Quantity', 'Line Total','Sales Channel Order Id']]

final['Normal'][(final['SKU'].isin(pro_exp.Normal_product)) & (final['Normal'].isnull())] = final['SKU']

search_sku = set((final.SKU[final['Normal'].isnull()]))
print ("Total No. of SKU for which Normal Not Found", set((final.SKU[final['Normal'].isnull()])))
for sku in search_sku:
  new_sku = input("Type Normal value for "+sku+" ")
  final.Normal[final['SKU'] == sku] = new_sku

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


Total No. of SKU for which Normal Not Found {'BM206289', 'BM204995', 'BM161592', 'BM207592', 'SDT-BUF-21515', 'SH8760', 'BM206082'}
Type Normal value for BM206289 BM206289


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Type Normal value for BM204995 BM204995
Type Normal value for BM161592 BM161592
Type Normal value for BM207592 BM207592
Type Normal value for SDT-BUF-21515 SDT-BUF-21515
Type Normal value for SH8760 NAU-SH8760
Type Normal value for BM206082 BM206082


In [32]:
vendor = pd.read_excel('Import List (5).xlsx')

vendor['Retail Channel SKU'] = vendor['Retail Channel SKU'].astype(str)
vendor['Vendor/ Mask SKU'] = vendor['Vendor/ Mask SKU'].astype(str)
vendor['Vendor'] = vendor['Vendor'].astype(str)

vendor.rename(columns={'Retail Channel SKU':'Normal', 'Vendor':'Vendor_Import'}, inplace=True)
# mask_vendor.rename(columns={'SKU':'Normal', 'Vendor':'Vendor_Import'}, inplace=True)

final['Normal'] = final['Normal'].astype(str)

store = final.merge(vendor, on='Normal', how='left')

store['Vendor'] = store['Vendor_Import']

store = store[['Sales Channel', 'Sales Order Id','Ship To Name','SKU' ,'Normal', 'Vendor','Quantity', 'Line Total','Sales Channel Order Id']]
final = store

vendor.rename(columns={'Normal':'Retail Channel SKU', 'Vendor/ Mask SKU':'Normal', 'Vendor_Import':'Vendor_Import_2'}, inplace=True)
store_new = final.merge(vendor, on='Normal', how='left')
store_new['Vendor'].fillna(store_new['Vendor_Import_2'])

import_file = store_new[store_new.Vendor.isnull() != True]

pro_exp.rename(columns={'SKU':'Masked SKU', 'Normal_product':'Normal', 'Vendor':'Vendor_f'}, inplace=True)

total = store_new.merge(pro_exp, on='Normal', how='left')
total['Vendor'] = total['Vendor'].fillna(total['Vendor_f'])

search_vendor = set((total.Normal[total.Vendor.isnull()]))
print ("Total No. of Normal for which Vendor Not Found", search_vendor)

for normal in search_vendor:
  new_vendor = input("Type vendor value for "+normal+" ")
  total.Vendor[total['Normal'] == normal] = new_vendor


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Total No. of Normal for which Vendor Not Found {'BM206289', 'BM204995', 'BM161592', 'BM207592', 'SDT-BUF-21515', 'BM206082'}
Type vendor value for BM206289 FOA Wholesale


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Type vendor value for BM204995 Uttermost Private Label
Type vendor value for BM161592 Sagebrookhome
Type vendor value for BM207592 FOA Wholesale
Type vendor value for SDT-BUF-21515 Summit Distribution
Type vendor value for BM206082 Sagebrookhome


## Warehouse Vendor

In [0]:
ware_vendor = pd.read_csv('ware_house_ven.xlsx')
ware_vendor.rename(columns={'Vendor Name':'Vendor'}, inplace=True)
ware_vendor['ware_vendors'] = ware_vendor['Vendor']

ware_vendor['Vendor'] = ware_vendor['Vendor'].astype(str)
ware_vendor['Vendor'] = ware_vendor['Vendor'].str.strip()
total['Vendor']=total['Vendor'].astype(str)
total['Vendor']=total['Vendor'].str.strip()

ware_vendor_file = total.merge(ware_vendor, on='Vendor', how='left')
ware_vendor_file = ware_vendor_file[ware_vendor_file['ware_vendors'].isnull()!=True]


## Get Coustomize column

In [34]:
!pip install xlsxwriter

#Get Coustomize column
total_file = total[['Sales Channel', 'Sales Order Id', 'Ship To Name', 'SKU', 'Normal', 'Vendor', 'Quantity', 'Line Total', 'Sales Channel Order Id']].round(2)

import_file = import_file[['Sales Channel', 'Sales Order Id', 'Ship To Name', 'SKU', 'Normal', 'Vendor', 'Quantity', 'Line Total', 'Sales Channel Order Id']].round(2)

ware_vendor_file = ware_vendor_file[['Sales Channel', 'Sales Order Id', 'Ship To Name', 'SKU', 'Normal', 'Vendor', 'Quantity', 'Line Total', 'Sales Channel Order Id']].round(2)

     |████████████████████████████████| 143kB 48.7MB/s 


## DropShip Vendors

In [0]:
dropship = total_file
dropship_vendor = pd.read_csv('drop_ship_ven.xlsx')

dropship_file = dropship[(dropship['Vendor'].isin(dropship_vendor['Vendor Name'])) & (dropship['Sales Channel'].isin(['Open Sky', 'Casagear', 'Amazon FBM', 'Walmart']))]


## CloseOut File

In [0]:
closeout_sku = pd.read_excel('closeout_sku.xlsx')
closeout_sku['SKU'] = closeout_sku['SKU'].astype(str)

#Filter Non CloseOut From All Sales Category
import_file = import_file[~import_file['Normal'].isin(closeout_sku['SKU'])]
ware_vendor_file = ware_vendor_file[~ware_vendor_file['Normal'].isin(closeout_sku['SKU']) & ~ware_vendor_file['Normal'].isin(import_file['Normal'])]
dropship_file = dropship_file[~dropship_file['Normal'].isin(closeout_sku['SKU']) & ~dropship_file['Normal'].isin(import_file['SKU'])& ~dropship_file['Normal'].isin(ware_vendor_file['SKU'])]
closeout_file = total_file[(total_file['Normal'].isin(closeout_sku['SKU']))]

category_notfound_sku = total_file[(~total_file['Normal'].isin(closeout_file['Normal']) & ~total_file['Normal'].isin(import_file['Normal']) & ~total_file['Normal'].isin(ware_vendor_file['Normal']) & ~total_file['Normal'].isin(dropship_file['Normal']))]

## Update Selling Type

In [37]:
search_type = set(category_notfound_sku.Normal)

print ("Total No. of Normal for which Selling Type not found", search_type)

for normal_sku in search_type:

  print (category_notfound_sku[category_notfound_sku['Normal']==normal_sku])
  sell_type = int(input("Select Selling Type for {}: 1-Import, 2-WareHouse, 3-DropShip, 4-CloseOut".format(normal_sku)))
#Import
  if (sell_type==1):
      import_file = pd.concat([import_file, category_notfound_sku[category_notfound_sku['Normal']==normal_sku]])
#WareHouse
  elif (sell_type==2): 
    ware_vendor_file = pd.concat([ware_vendor_file, category_notfound_sku[category_notfound_sku['Normal']==normal_sku]]) 
#DropShip
  elif (sell_type==3):
    dropship_file = pd.concat([dropship_file, category_notfound_sku[category_notfound_sku['Normal']==normal_sku]])
#CloseOut
  elif (sell_type==4):
    closeout_file = pd.concat([closeout_file, category_notfound_sku[category_notfound_sku['Normal']==normal_sku]])

Total No. of Normal for which Selling Type not found {'SDT-BUF-21515', 'SVN-SG-MRBD-043'}
   Sales Channel  Sales Order Id  ... Line Total       Sales Channel Order Id
63       Walmart        650833.0  ...      14.45  3803743024620-2752046753342

[1 rows x 9 columns]
Select Selling Type for SDT-BUF-21515: 1-Import, 2-WareHouse, 3-DropShip, 4-CloseOut3
    Sales Channel  Sales Order Id  ... Line Total Sales Channel Order Id
417    Amazon FBA             NaN  ...        4.9    113-3984442-4037026

[1 rows x 9 columns]
Select Selling Type for SVN-SG-MRBD-043: 1-Import, 2-WareHouse, 3-DropShip, 4-CloseOut1


## Get previous month data

In [0]:
ds_total = ds_total[(ds_total['Date'] >= start_date) & (ds_total['Date'] <= end_date)]
ds_import = ds_import[(ds_import['Date'] >= start_date) & (ds_import['Date'] <= end_date)]
ds_warehouse = ds_warehouse[(ds_warehouse['Date'] >= start_date) & (ds_warehouse['Date'] <= end_date)]
ds_dropship = ds_dropship[(ds_dropship['Date'] >= start_date) & (ds_dropship['Date'] <= end_date)]
ds_closeout = ds_closeout[(ds_closeout['Date'] >= start_date) & (ds_closeout['Date'] <= end_date)]

In [39]:
ds_total['Line Total'].sum()

228342.77999999997

## Update Closeout movement

In [40]:
take_closeout_temp = closeout_file
take_closeout_temp['Date'] = receive_date
cols = list(take_closeout_temp.columns)
cols = [cols[-1]] + cols[:-1]
take_closeout_temp = take_closeout_temp[cols]
take_closeout_temp.columns = ds_closeout.columns
take_closeout_temp_2 = pd.concat([ds_closeout, take_closeout_temp],ignore_index=True)
take_closeout_temp_2['Quantity'] = take_closeout_temp_2['Quantity'].astype(float)
take_closeout_temp_2.groupby(['Normal SKU']).sum().reset_index()['Normal SKU'].count(), take_closeout_temp_2.groupby(['Normal SKU']).sum().reset_index()
take_closeout_temp_c, take_closeout_temp_s = take_closeout_temp_2.groupby(['Normal SKU']).sum().reset_index()['Normal SKU'].count(), take_closeout_temp_2.groupby(['Normal SKU']).sum().reset_index()['Quantity'].sum()
take_closeout_temp_c, take_closeout_temp_s

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(96, 149.0)

## Download Daily sale File

In [0]:
from google.colab import files

writer = pd.ExcelWriter('Daily sales '+receive_date+'.xlsx', engine='xlsxwriter')

total_file.to_excel(writer, sheet_name='total_sale')
import_file.to_excel(writer, sheet_name='import_sale')
ware_vendor_file.to_excel(writer, sheet_name='warehouse_sale')
dropship_file.to_excel(writer, sheet_name='dropship_sale')
closeout_file.to_excel(writer, sheet_name='closeout_file')

writer.save()
writer.close()

files.download('Daily sales '+receive_date+'.xlsx')

## Create Pivot Channel, vendor, SKU wise for Total, Import, Warehouse, Dropship and CloseOut

In [0]:
ds_warehouse['Vendor'] = ds_warehouse['Vendor'].astype(str)
ds_warehouse['Vendor'] = ds_warehouse['Vendor'].str.strip()
ds_dropship['Vendor'] = ds_dropship['Vendor'].astype(str)
ds_dropship['Vendor'] = ds_dropship['Vendor'].str.strip()

ds_total['Line Total'] = ds_total['Line Total'].astype(float)
ds_import['Line Total'] = ds_import['Line Total'].astype(float)
ds_warehouse['Line Total'] = ds_warehouse['Line Total'].astype(float)
ds_dropship['Line Total'] = ds_dropship['Line Total'].astype(float)
ds_closeout['Line Total'] = ds_closeout['Line Total'].astype(float)

In [0]:
#total
channel_total = ds_total.groupby([ds_total['Sales Channel'].rename('Channel')]).agg({'Line Total': 'sum'}).rename(columns={'Line Total':'Month total'}).reset_index().reset_index()
vendor_total = ds_total.groupby([ds_total['Vendor'].rename('Vendor')]).agg({'Line Total': 'sum'}).rename(columns={'Line Total':'Month total'}).reset_index()
channel_total_today = total.groupby([total['Sales Channel'].rename('Channel')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal SKU':'Total_Orders'}).reset_index()
vendor_total_today = total.groupby([total['Vendor'].rename('Vendor')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal SKU':'Total_Orders'}).reset_index()
sku_total_today = total.groupby([total['Normal'].rename('SKU')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal':'Total_Orders'}).reset_index()
sku_total_today = sku_total_today[['SKU', 'Quantity', 'Line Total']].sort_values(['Line Total'], ascending=False).fillna(0).round(2)

#import
channel_import = ds_import.groupby([ds_import['Sales Channel'].rename('Channel')]).agg({'Line Total': 'sum'}).rename(columns={'Line Total':'Month total'}).reset_index()
vendor_import = ds_import.groupby([ds_import['Vendor'].rename('Vendor')]).agg({'Line Total': 'sum'}).rename(columns={'Line Total':'Month total'}).reset_index()
channel_import_today = import_file.groupby([import_file['Sales Channel'].rename('Channel')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal SKU':'Total_Orders'}).reset_index()
vendor_import_today = import_file.groupby([import_file['Vendor'].rename('Vendor')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal SKU':'Total_Orders'}).reset_index()
sku_import_today = import_file.groupby([import_file['Normal'].rename('SKU') , import_file['Vendor'].rename('Vendor')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal':'Total_Orders'}).reset_index()
sku_import_today = sku_import_today[['SKU', 'Quantity', 'Line Total', 'Vendor']].sort_values(['Line Total'], ascending=False).fillna(0).round(2)

#warehouse
channel_warehouse = ds_warehouse.groupby([ds_warehouse['Sales Channel'].rename('Channel')]).agg({'Line Total': 'sum'}).rename(columns={'Line Total':'Month total'}).reset_index()
vendor_warehouse = ds_warehouse.groupby([ds_warehouse['Vendor'].rename('Vendor')]).agg({'Line Total': 'sum'}).rename(columns={'Line Total':'Month total'}).reset_index()
channel_warehouse_today = ware_vendor_file.groupby([ware_vendor_file['Sales Channel'].rename('Channel')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal SKU':'Total_Orders'}).reset_index()
vendor_warehouse_today = ware_vendor_file.groupby([ware_vendor_file['Vendor'].rename('Vendor')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal SKU':'Total_Orders'}).reset_index()
sku_warehouse_today = ware_vendor_file.groupby([ware_vendor_file['Normal'].rename('SKU')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal':'Total_Orders'}).reset_index()
sku_warehouse_today = sku_warehouse_today[['SKU', 'Quantity', 'Line Total']].sort_values(['Line Total'], ascending=False).fillna(0).round(2)

#dropship
channel_dropship = ds_dropship.groupby([ds_dropship['Sales Channel'].rename('Channel')]).agg({'Line Total': 'sum'}).rename(columns={'Line Total':'Month total'}).reset_index()
vendor_dropship = ds_dropship.groupby([ds_dropship['Vendor'].rename('Vendor')]).agg({'Line Total': 'sum'}).rename(columns={'Line Total':'Month total'}).reset_index()
channel_dropship_today = dropship_file.groupby([dropship_file['Sales Channel'].rename('Channel')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal SKU':'Total_Orders'}).reset_index()
vendor_dropship_today = dropship_file.groupby([dropship_file['Vendor'].rename('Vendor')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal SKU':'Total_Orders'}).reset_index()
sku_dropship_today = dropship_file.groupby([dropship_file['Normal'].rename('SKU')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal':'Total_Orders'}).reset_index()
sku_dropship_today = sku_dropship_today[['SKU', 'Quantity', 'Line Total']].sort_values(['Line Total'], ascending=False).fillna(0).round(2)

#CloseOut
channel_closeout = ds_closeout.groupby([ds_closeout['Sales Channel'].rename('Channel')]).agg({'Line Total': 'sum'}).rename(columns={'Line Total':'Month total'}).reset_index()
vendor_closeout = ds_closeout.groupby([ds_closeout['Vendor'].rename('Vendor')]).agg({'Line Total': 'sum'}).rename(columns={'Line Total':'Month total'}).reset_index()
channel_closeout_today = closeout_file.groupby([closeout_file['Sales Channel'].rename('Channel')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal SKU':'Total_Orders'}).reset_index()
vendor_closeout_today = closeout_file.groupby([closeout_file['Vendor'].rename('Vendor')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal SKU':'Total_Orders'}).reset_index()
sku_closeout_today = closeout_file.groupby([closeout_file['Normal'].rename('SKU')]).agg({'Normal':'count', 'Quantity':'sum','Line Total': 'sum'}).rename(columns={'Normal':'Total_Orders'}).reset_index()
sku_closeout_today = sku_closeout_today[['SKU', 'Quantity', 'Line Total']].sort_values(['Line Total'], ascending=False).fillna(0).round(2)

In [0]:
# channel_total_p = channel_total.merge(channel_total_today, on='Channel', how='left').fillna(0).round(2)
#total
channel_total_p = channel_total.merge(channel_total_today, on='Channel', how='outer').fillna(0).round(2)
channel_total_p['Month total'] = channel_total_p['Month total']+channel_total_p['Line Total']
channel_total_p = channel_total_p[['Channel', 'Normal', 'Quantity', 'Line Total', 'Month total']].fillna(0).round(2)
vendor_total_p = vendor_total.merge(vendor_total_today, on='Vendor', how='outer').fillna(0).round(2).sort_values(['Line Total'], ascending=False)
vendor_total_p['Month total'] = vendor_total_p['Month total']+vendor_total_p['Line Total']
vendor_total_p = vendor_total_p[['Vendor', 'Quantity', 'Line Total', 'Month total']].fillna(0).round(2)

#import
channel_import_p = channel_import.merge(channel_import_today, on='Channel', how='outer').fillna(0).round(2)
channel_import_p['Month total'] = channel_import_p['Month total']+channel_import_p['Line Total']
channel_import_p = channel_import_p[['Channel', 'Normal', 'Quantity', 'Line Total', 'Month total']].fillna(0).round(2)
vendor_import_p = vendor_import.merge(vendor_import_today, on='Vendor', how='outer').fillna(0).round(2).sort_values(['Line Total'], ascending=False)
vendor_import_p['Month total'] = vendor_import_p['Month total']+vendor_import_p['Line Total']
vendor_import_p = vendor_import_p[['Vendor', 'Quantity', 'Line Total', 'Month total']].fillna(0).round(2)

#warehouse
channel_ware_p = channel_warehouse.merge(channel_warehouse_today, on='Channel', how='outer').fillna(0).round(2)
channel_ware_p['Month total'] = channel_ware_p['Month total']+channel_ware_p['Line Total']
channel_ware_p = channel_ware_p[['Channel', 'Normal', 'Quantity', 'Line Total', 'Month total']].fillna(0).round(2)
vendor_ware_p = vendor_warehouse.merge(vendor_warehouse_today, on='Vendor', how='outer').fillna(0).round(2).sort_values(['Line Total'], ascending=False)
vendor_ware_p['Month total'] = vendor_ware_p['Month total']+vendor_ware_p['Line Total']
vendor_ware_p = vendor_ware_p[['Vendor', 'Quantity', 'Line Total', 'Month total']].fillna(0).round(2)

#dropship
channel_drop_p = channel_dropship.merge(channel_dropship_today, on='Channel', how='outer').fillna(0).round(2)
channel_drop_p['Month total'] = channel_drop_p['Month total']+channel_drop_p['Line Total']
channel_drop_p = channel_drop_p[['Channel', 'Normal', 'Quantity', 'Line Total', 'Month total']].fillna(0).round(2)
vendor_drop_p = vendor_dropship.merge(vendor_dropship_today, on='Vendor', how='outer').fillna(0).round(2).sort_values(['Line Total'], ascending=False)
vendor_drop_p['Month total'] = vendor_drop_p['Month total']+vendor_drop_p['Line Total']
vendor_drop_p = vendor_drop_p[['Vendor', 'Quantity', 'Line Total', 'Month total']].fillna(0).round(2)

#CloseOut
channel_close_p = channel_closeout.merge(channel_closeout_today, on='Channel', how='outer').fillna(0).round(2)
channel_close_p['Month total'] = channel_close_p['Month total']+channel_close_p['Line Total']
channel_close_p = channel_close_p[['Channel', 'Normal', 'Quantity', 'Line Total', 'Month total']].fillna(0).round(2)
vendor_close_p = vendor_closeout.merge(vendor_closeout_today, on='Vendor', how='outer').fillna(0).round(2).sort_values(['Line Total'], ascending=False)
vendor_close_p['Month total'] = vendor_close_p['Month total']+vendor_close_p['Line Total']
vendor_close_p = vendor_close_p[['Vendor', 'Quantity', 'Line Total', 'Month total']].fillna(0).round(2)

## "temperory" Adding nov closeout sale to current month

In [0]:
lm_closeout_sale = pd.read_excel('last_month_sale.xlsx',4)
nov_closeout_sale = pd.read_excel('nov_sale.xlsx',4)

lm_closeout_sale.columns = ds_closeout.columns
nov_closeout_sale.columns = ds_closeout.columns
nov_closeout_sale["Date"] = pd.to_datetime(nov_closeout_sale["Date"])

lm_closeout_sale_temp1 = pd.concat([nov_closeout_sale, lm_closeout_sale, ds_closeout], ignore_index=True)[['Sales Channel', 'Vendor', 'Line Total']]
lm_closeout_sale_temp1 = lm_closeout_sale_temp1.rename(columns = {"Sales Channel": "Channel", "Line Total":"sales_lm_tod"})
# lm_closeout_sale_temp1
# lm_closeout_sale_temp1.groupby('Channel').sum().reset_index()
channel_close_p = pd.merge(channel_close_p, (lm_closeout_sale_temp1.groupby('Channel').sum().reset_index()), on='Channel', how='outer').fillna(0).round(2).sort_values(['Line Total', 'Month total', 'sales_lm_tod'], ascending=False)
vendor_close_p = pd.merge(vendor_close_p, lm_closeout_sale_temp1.groupby('Vendor').sum().reset_index(), on='Vendor', how='outer').fillna(0).round(2).sort_values(['Line Total', 'Month total', 'sales_lm_tod'], ascending=False)

channel_close_p['sales_lm_tod'] = channel_close_p['sales_lm_tod'] + channel_close_p['Line Total']
vendor_close_p['sales_lm_tod'] = vendor_close_p['sales_lm_tod'] + vendor_close_p['Line Total']

##Average Sale Growth w.r.t. last month

In [47]:
lm_sale = pd.read_excel('last_month_sale.xlsx')
lmavg_channel = lm_sale.groupby('Sales Channel').sum().reset_index()[['Sales Channel', 'Line Total']]
lmavg_channel = lmavg_channel.rename(columns={'Sales Channel':'Channel'})
avg_fact = int(receive_date.split('-')[-1])

lmavg_channel['lm_avg'] = (lmavg_channel['Line Total']/30).round(2)
lm_total_avg = lmavg_channel['Line Total'].sum()/30
channel_total_p['avg_sale'] = (channel_total_p['Month total']/avg_fact).round(2)
channel_total_pn = channel_total_p.merge(lmavg_channel[['Channel', 'lm_avg']], on='Channel', how='left')
channel_total_pn['avg_growth'] = ((channel_total_pn['avg_sale']-channel_total_pn['lm_avg'])/channel_total_pn['lm_avg']).round(5)
channel_total_pn

,Channel,Normal,Quantity,Line Total,Month total,avg_sale,lm_avg,avg_growth
0,Amazon FBA,6.0,6.0,93.27,426.58,85.32,109.78,-0.22281
1,Amazon FBM,52.0,63.0,5874.05,29156.98,5831.40,4743.76,0.22928
2,Amazon Vendor CA,0.0,0.0,0.00,2516.01,503.20,726.64,-0.30750
3,Amazon Vendor DS,19.0,20.0,2019.62,5922.03,1184.41,776.24,0.52583
4,Amazon Vendor US,0.0,0.0,0.00,54683.25,10936.65,5949.35,0.83829
5,BENZARA WHOLESALE,0.0,0.0,0.00,133.50,26.70,174.66,-0.84713
6,Casagear,0.0,0.0,0.00,1672.26,334.45,562.13,-0.40503
7,Doba,0.0,0.0,0.00,87.72,17.54,61.49,-0.71475
8,Hayneedle,8.0,11.0,2455.06,5599.76,1119.95,813.08,0.37742
9,Home Depot.com,21.0,28.0,2800.41,6418.54,1283.71,911.12,0.40894


In [48]:
total_Sale_amount = channel_total_pn['Line Total'].sum()
channel_total_pn['contribution'] = ((channel_total_pn['Line Total']/total_Sale_amount)*100).round(2)

rowtotal = channel_total_pn.sum()
rowtotal.Channel = 'Total Sales'
channel_total_pn = channel_total_pn.append(rowtotal, ignore_index=True).sort_values(by='Line Total', ascending=False)
channel_total_pn['lm_avg'][channel_total_pn['Channel']=='Total Sales'] = lm_total_avg
channel_total_pn['avg_growth'][channel_total_pn['Channel']=='Total Sales'] = ((channel_total_pn['avg_sale']-lm_total_avg)/lm_total_avg).round(5)

channel_total_p = channel_total_pn[['Channel', 'Normal', 'Quantity', 'Line Total', 'Month total', 'avg_sale', 'lm_avg', 'avg_growth']]
channel_total_p

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Channel,Normal,Quantity,Line Total,Month total,avg_sale,lm_avg,avg_growth
22,Total Sales,421.0,512.0,61570.38,289913.16,57982.66,54908.908667,0.05598
21,Wayfair,227.0,279.0,34033.28,130905.22,26181.04,24399.110000,0.07303
15,Overstock,32.0,42.0,6367.11,21783.33,4356.67,3738.620000,0.16532
1,Amazon FBM,52.0,63.0,5874.05,29156.98,5831.40,4743.760000,0.22928
9,Home Depot.com,21.0,28.0,2800.41,6418.54,1283.71,911.120000,0.40894
8,Hayneedle,8.0,11.0,2455.06,5599.76,1119.95,813.080000,0.37742
3,Amazon Vendor DS,19.0,20.0,2019.62,5922.03,1184.41,776.240000,0.52583
14,Open Sky,19.0,22.0,1924.56,8280.03,1656.01,2753.440000,-0.39857
11,Macys,3.0,5.0,1797.01,4841.84,968.37,665.360000,0.45541
16,Target.com,8.0,8.0,1265.31,3048.03,609.61,3446.320000,-0.82311


## Merge SKUs wise sale to vendor and channel

In [0]:
# sku_total_today
sku_total_temp = total_file.groupby([total_file['Normal'].rename('SKU'),total_file['Sales Channel'].rename('Channel'), total_file['Vendor'].rename('Vendor')]).agg({'Quantity':'sum','Line Total': 'sum'}).rename(columns={"Line Total": "sale"}).sort_values(by=['sale', 'SKU'], ascending=False).reset_index()
sku_import_temp = import_file.groupby([import_file['Normal'].rename('SKU'), import_file['Vendor'].rename('Vendor')]).agg({'Quantity':'sum','Line Total': 'sum'}).rename(columns={"Line Total": "sale"}).sort_values(by=['sale'], ascending=False).reset_index()
sku_ware_temp = ware_vendor_file.groupby([ware_vendor_file['Normal'].rename('SKU'),ware_vendor_file['Sales Channel'].rename('Channel'), ware_vendor_file['Vendor'].rename('Vendor')]).agg({'Quantity':'sum','Line Total': 'sum'}).rename(columns={"Line Total": "sale"}).sort_values(by=['sale', 'SKU'], ascending=False).reset_index()
sku_dropship_temp = dropship_file.groupby([dropship_file['Normal'].rename('SKU'),dropship_file['Sales Channel'].rename('Channel'), dropship_file['Vendor'].rename('Vendor')]).agg({'Quantity':'sum','Line Total': 'sum'}).rename(columns={"Line Total": "sale"}).sort_values(by=['sale', 'SKU'], ascending=False).reset_index()
sku_closeout_temp = closeout_file.groupby([closeout_file['Normal'].rename('SKU'),closeout_file['Sales Channel'].rename('Channel'), closeout_file['Vendor'].rename('Vendor')]).agg({'Quantity':'sum','Line Total': 'sum'}).rename(columns={"Line Total": "sale"}).sort_values(by=['sale', 'SKU'], ascending=False).reset_index()

In [50]:
sku_total_temp.head()

,SKU,Channel,Vendor,Quantity,sale
0,HLD-5265DTBCW7,Amazon FBM,Hillsdale Furniture,1.0,1747.85
1,BM207592,Wayfair,FOA Wholesale,2.0,1740.16
2,UPT-37910,Wayfair,Aaina Craft,25.0,1152.00
3,FOA-CM3324BK-SC-2PK,Hayneedle,FOA Wholesale,3.0,863.34
4,FOA-CM3321PT-5PK,Wayfair,FOA Wholesale,2.0,835.28


## Convert File to Excel

In [0]:
writer = pd.ExcelWriter('channel Performance '+receive_date+'.xlsx', engine='xlsxwriter')

channel_total_p.to_excel(writer, sheet_name='total_c')
vendor_total_p.to_excel(writer, sheet_name='total_v')
sku_total_temp.to_excel(writer, sheet_name='total_s')
writer.save()

writer = pd.ExcelWriter('Import Performance '+receive_date+'.xlsx', engine='xlsxwriter')

channel_import_p.to_excel(writer, sheet_name='import_c')
vendor_import_p.to_excel(writer, sheet_name='import_v')
sku_import_temp.to_excel(writer, sheet_name='import_s')
writer.save()

writer = pd.ExcelWriter('WareHouse Performance '+receive_date+'.xlsx', engine='xlsxwriter')

channel_ware_p.to_excel(writer, sheet_name='ware_c')
vendor_ware_p.to_excel(writer, sheet_name='ware_v')
sku_ware_temp.to_excel(writer, sheet_name='ware_s')
writer.save()

writer = pd.ExcelWriter('Dropship Performance '+receive_date+'.xlsx', engine='xlsxwriter')

channel_drop_p.to_excel(writer, sheet_name='drop_c')
vendor_drop_p.to_excel(writer, sheet_name='drop_v')
sku_dropship_temp.to_excel(writer, sheet_name='drop_s')
writer.save()

writer = pd.ExcelWriter('CloseOut Performance '+receive_date+'.xlsx', engine='xlsxwriter')

channel_close_p.to_excel(writer, sheet_name='close_c')
vendor_close_p.to_excel(writer, sheet_name='close_v')
sku_closeout_temp.to_excel(writer, sheet_name='close_s')
writer.save()

writer.close()

## Download Sale File Order Wise

In [0]:
from google.colab import files
files.download('channel Performance '+receive_date+'.xlsx')
files.download('Import Performance '+receive_date+'.xlsx')
files.download('WareHouse Performance '+receive_date+'.xlsx')
files.download('Dropship Performance '+receive_date+'.xlsx')
files.download('CloseOut Performance '+receive_date+'.xlsx')